In [ ]:
import tensorflow as tf
from tensorflow import convert_to_tensor, string
from tensorflow.keras.layers import TextVectorization, Embedding, Layer
from tensorflow.data import Dataset
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
output_sequence_length = 5
vocab_size = 10
sentences = [["I am a robot"], ["you too robot"]]
sentence_data = Dataset.from_tensor_slices(sentences)
vectorize_layer = TextVectorization(output_sequence_length=output_sequence_length, max_tokens=vocab_size)

vectorize_layer.adapt(sentence_data)

word_tensors = convert_to_tensor(sentences, dtype=tf.string)

vectorized_words = vectorize_layer(word_tensors)

print(vectorize_layer.get_vocabulary())
print(vectorized_words)

In [ ]:
output_length = 6
word_embedding_layer = Embedding(vocab_size, output_length)
embedded_words = word_embedding_layer(vectorized_words)
print(embedded_words)

In [ ]:
position_embedding_layer = Embedding(output_sequence_length, output_length)
position_indices = tf.range(output_sequence_length)
print(position_indices)
embedded_indices = position_embedding_layer(position_indices)
print(embedded_indices)

In [ ]:
final_output_embedding = embedded_words + embedded_indices
print("Final output: ", final_output_embedding)

In [ ]:
class PositionalEmbeddingLayer(Layer):
    def __init__(self, seq_length, vocab_size, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.word_embedding_layer = Embedding(input_dim=vocab_size, output_dim=output_dim)
        self.position_embedding_layer = Embedding(input_dim=seq_length, output_dim=output_dim)

    def call(self, inputs):
        position_indices = tf.range(tf.shape(inputs)[-1])
        embedded_words = self.word_embedding_layer(inputs)
        embedded_indices = self.position_embedding_layer(position_indices)
        return embedded_words + embedded_indices

In [ ]:
my_embedding_layer = PositionalEmbeddingLayer(output_sequence_length, vocab_size, output_length)
embedding_layer_output = my_embedding_layer(vectorized_words)
print("Output from my_embedding_layer= ", embedding_layer_output)

In [ ]:
class PositionalEmbeddingFixedWeightsLayer(Layer):
    def __init__(self, seq_length, vocab_size, output_dim, **kwargs):
        super().__init__(**kwargs)
        word_embedding_weights = self.get_position_encoding(vocab_size, output_dim)
        position_embedding_weights = self.get_position_encoding(seq_length, output_dim)
        self.word_embedding_layer = Embedding(input_dim=vocab_size, output_dim=output_dim, weights=[word_embedding_weights], trainable=False)
        self.position_embedding_layer = Embedding(input_dim=seq_length, output_dim=output_dim, weight=[position_embedding_weights], trainable=False)

    def get_position_encoding(self, seq_len, d, n=10000):
        P = np.zeros((seq_len, d))
        for k in range(seq_len):
            for i in np.arange(int(d/2)):
                denominator = np.power(n, 2*i/d)
                P[k, 2*i] = np.sin(k/denominator)
                P[k, 2*i + 1] = np.cos(k/denominator)
        return P

    def call(self, inputs):
        position_indices = tf.range(tf.shape(inputs)[-1])
        embedded_words = self.word_embedding_layer(inputs)
        embedded_indices = self.position_embedding_layer(position_indices)
        return embedded_words + embedded_indices